In [1]:
import pandas as pd
# improt company data with domains
df = pd.read_csv('data/data.csv')

In [2]:
df

,Company,domain,industry
0,AAFCPAs,aafcpa.com,Accounting
1,Trans World Airlines,7online.com,Airlines/Aviation
2,2 Step Divorces,2StepDivorces.com,Alternative Dispute Resolution
3,3D Cannabis Center,3DMMC.com,Alternative Medicine
4,10 Forward Productions,10forwardproductions.com,Animation
5,47 Brand,47brand.com,Apparel & Fashion
6,4240 Architecture,4240architecture.com,Architecture & Planning
7,Carina Picture Frame Inc,123frame.net,Arts and Crafts
8,1-800 Radiator & A/C,1800radiator.com,Automotive
9,Idaho Air National Guard,124wg.ang.af.mil,Aviation & Aerospace


In [ ]:
#Enrich the list of domains from the sample dataset with industies from ZoomInfo

#Generate access token (valid for 1 hour)

#import libraries and declare a dictionary with ZI-specific fields
#run 'pip install PyJWT' before importing, use the right kernel or call 
#!pip install PyJWT

import jwt
from datetime import datetime, date, time, timedelta
import uuid

pc ='YourPartnerCode'
zipayload = {
'ziPartnerCode':pc
 }

In [ ]:
#set the creation date and the expiration date (up to 1 hour)

utc0 = datetime(1970,1,1)
issueTime = datetime.utcnow()
iat = issueTime - utc0
iat = int(iat.total_seconds())
exp = issueTime - utc0
exp = exp + timedelta(minutes=60)
exp = int(exp.total_seconds())

In [ ]:
#create a dictionary with the required payload information

payload = {
 'iss':'zoominfo',\
 'iat': iat,\
 'jti': str(uuid.uuid4()),\
 'ziPayLoad':zipayload,\
 'exp': exp
 }

In [ ]:
#encode the data and generate the JWT (encoded will have the value of JWT)

password = 'yourpassword' #password
headers={'Accept': 'application/xml', 'Content-type': 'application/xml'}
encoded = jwt.encode(payload, password, algorithm='HS256', headers = headers)


In [ ]:
#convert from bytes to string
encoded_str = encoded.decode()

In [ ]:
encoded_str

In [ ]:
#Make a post request to ZI API to get the access token, based on the JWT
#Run 'pip3 install httplib2' in your terminal window before importing
import httplib2
import urllib

http = httplib2.Http()


body = '<?xml version="1.0" encoding="utf-8"?><TokenPostInput xmlns="http://partnerapi.zoominfo.com/partnerapistatic/xsd/V1/TokenPostInput.xsd"><pc>YourKey</pc><ziToken>YourToken</ziToken></TokenPostInput>'


url = "https://partnerapi.zoominfo.com/partnerapi/v2/token"

response, content = http.request(url, method = "POST", body = body, headers = headers)

print(content.decode())
print('Status: ' + response['status'])



In [ ]:
#create CompanyDetail search request
import requests

domains = df['domain'].iloc[37000:38000]
domains = domains.tolist()
urls = []

#add www to the domain names
for domain in domains:
    urls.append(('www.%s') % (domain))
    
len(domains)

In [ ]:
enriched_domains = {}
try: 
    for url in urls:
        x = ("https://partnerapi.zoominfo.com/partnerapi/v4/company/detail?pc=YourPartnerCode&key=YourKey&CompanyDomain=%s&outputType=json&outputFieldOptions=companyTopLevelIndustry") % (url)
        call = requests.get(x)
        json_data = call.json()  
        extract = json_data['companyDetailResponse']
        enriched_domains[url] = extract
except ConnectionResetError:
    print('Handle Exception')




In [ ]:
type(enriched_domains)

In [ ]:
print(enriched_domains)

In [ ]:
#convert dataframe to a list and remove column headers
dict2ls = pd.DataFrame(enriched_domains)

dict2ls = dict2ls.transpose()
dict2ls.to_csv('data/enriched/enriched_ZI_domains.csv', sep = ',')